In [1]:
import warnings
from pathlib import Path

import iris
import numpy as np
from cf_units import Unit
from tqdm.notebook import tqdm as tqdm

In [2]:
from aeolus.coord import interp_cube_from_height_to_pressure_levels
from aeolus.synthobs import read_spectral_bands

In [3]:
from util_commons import PLANETS, SUITES
from util_mypaths import path_to_data_umserve, path_to_processed

In [4]:
warnings.filterwarnings("ignore", module="iris")
warnings.filterwarnings("ignore", module="aeolus")

In [5]:
# Parameters
metallicity = "1x solar"
tgt_plevs = np.logspace(7, 2, 65)  # target pressure levels

In [6]:
# Read spectral bands from the SOCRATES spectral file
# (band numbering and bounds are the same for HAT-P-11b, HD 189733b, HD 209458b and WASP-17b)
path_to_star_spectrum_lw = SUITES["hatp11b"]["equilibrium"][metallicity]["dir_for_star_spectrum_lw"]
socrates_bands = read_spectral_bands(path_to_star_spectrum_lw)
# SOCRATES bands corresponding to the center of the Spitzer/IRAC 3.6, 4.5, 5.8 and 8.0 micrometer channels,
# and an extra band for 10.5 micrometers containing an NH3 feature
BANDS = {
    "3.6": {
        "species": "ch4",
        "index": 278,
        "bounds_socrates": (socrates_bands[277][1], socrates_bands[277][2]),
        "bounds_spitzer_irac": (3.081060, 4.010380),
    },
    "4.5": {
        "species": "co",
        "index": 223,
        "bounds_socrates": (socrates_bands[222][1], socrates_bands[222][2]),
        "bounds_spitzer_irac": (3.722490, 5.221980),
    },
    "5.8": {
        "species": "h2o",
        "index": 173,
        "bounds_socrates": (socrates_bands[172][1], socrates_bands[172][2]),
        "bounds_spitzer_irac": (4.744210, 6.622510),
    },
    "8.0": {
        "species": "ch4",
        "index": 125,
        "bounds_socrates": (socrates_bands[124][1], socrates_bands[124][2]),
        "bounds_spitzer_irac": (6.151150, 9.728750),
    },
    "10.5": {
        "species": "nh3",
        "index": 96,
        "bounds_socrates": (socrates_bands[95][1], socrates_bands[95][2]),
        "bounds_spitzer_irac": (socrates_bands[95][1] * 1e6, socrates_bands[95][2] * 1e6),
    },
}

In [7]:
vrbls = {}
for planet in tqdm(PLANETS.keys()):
    vrbls[planet] = {}
    for exp in ["equilibrium", "kinetics"]:
        vrbls[planet][exp] = {}
        # Load the last time step for air_pressure
        fname = f"{SUITES[planet][exp][metallicity]['rose_suite']}.nc"
        path_to_merged = SUITES[planet][exp][metallicity]["dir_for_merged"]
        air_pressure = iris.load_cube(str(path_to_merged / fname), "air_pressure")[-1, ...]
        # Load all bands of flux_contribution_function_lw
        path_to_con_func = SUITES[planet][exp][metallicity]["dir_for_flux_contribution_function_lw"]
        con_func_all_bands = iris.load_cube(str(path_to_con_func))
        for band in BANDS.keys():
            vrbls[planet][exp][band] = {}
            band_in_nm = int(float(band) * 1e3)
            # Select by band
            con_func = con_func_all_bands[BANDS[band]["index"], ...]
            con_func.rename(f"flux_contribution_function_on_theta_levels_at_{band_in_nm}_nm")
            con_func.units = Unit("W m-2")
            # Relevel to pressure levels
            con_func_plevs = interp_cube_from_height_to_pressure_levels(
                con_func, air_pressure, tgt_plevs
            )
            con_func_plevs.rename(
                f"flux_contribution_function_on_pressure_levels_at_{band_in_nm}_nm"
            )
            con_func_plevs.units = Unit("W m-2")
            # Normalise contribution function by the maximum column value column-wise
            norm_con_func = con_func_plevs / con_func_plevs.collapsed(
                "air_pressure", iris.analysis.MAX
            )
            norm_con_func.rename(
                f"normalised_flux_contribution_function_on_pressure_levels_at_{band_in_nm}_nm"
            )
            norm_con_func.units = Unit("1")
            # Assemble data
            vrbls[planet][exp][band] = {
                "con_func": con_func,
                "con_func_plevs": con_func_plevs,
                "norm_con_func": norm_con_func,
            }

  0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# Save
for planet in PLANETS.keys():
    for exp in ["equilibrium", "kinetics"]:
        for band in BANDS.keys():
            band_in_nm = int(float(band) * 1e3)
            cl = [v for v in vrbls[planet][exp][band].values()]
            iris.save(
                cl,
                str(
                    path_to_processed
                    / planet
                    / exp
                    / "flux_contribution_function_lw_spitzer"
                    / f"{planet}_{exp[0:3]}_flux_con_func_{band_in_nm}_nm.nc"
                ),
            )